In [1]:
!pip install reportlab
!pip install mysql-connector-python
!pip install fpdf
!pip install --upgrade selenium

## Sql Executer code

In [ ]:
import mysql.connector
from tkinter import *
from tkinter import ttk 
from tkinter import messagebox
from tkinter import font
from tkinter import simpledialog
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors
from tkinter import Button
import getpass


def execute_query():
    query_choice = query_var.get()
    if query_choice == "1":
        delivery_date = input_date.get()
        if not delivery_date:
            messagebox.showwarning("Missing Input", "Please enter delivery date.")
            return
        query = f"SELECT d.DeliveryAgentID, COUNT(*) AS TotalDeliveries, SUM(CASE WHEN d.DeliverySuccess = 1 THEN 1 ELSE 0 END) AS SuccessfulDeliveries, AVG(CASE WHEN d.DeliverySuccess = 1 THEN 1 ELSE 0 END) AS SuccessRate FROM Delivery AS d WHERE d.DeliveryDate = '{delivery_date}' GROUP BY d.DeliveryAgentID;"
        query_desc = "Query Description: Total deliveries and success rate for a specific date with delivery agent"
        input_date_label.pack()
        input_date.pack()
    elif query_choice == "2":
        amount = input_amount.get()
        if not amount:
            messagebox.showwarning("Missing Input", "Please enter a specific amount.")
            return
        query = f"SELECT c.CustomerID, c.CustomerName, COUNT(rp.ReturnProductID) AS TotalProductsReturned FROM Customers AS c JOIN PaymentProfiles AS pp ON c.CustomerID = pp.CustomerID JOIN Payment AS p ON pp.PaymentProfileID = p.PaymentProfileID JOIN OrderDetails AS od ON p.OrderNum = od.OrderNum JOIN ReturnedProduct AS rp ON od.OrderNum = rp.OrderNum WHERE p.Amount > {amount} GROUP BY c.CustomerID, c.CustomerName;"
        query_desc = "Query Description: Total products returned by customers with payments over a specific amount"
        input_amount_label.pack()
        input_amount.pack()
    elif query_choice == "3":
        total_orders = input_orders.get()
        if not total_orders:
            messagebox.showwarning("Missing Input", "Please enter the amount of orders.")
            return
        query = f"SELECT c.CustomerID, c.CustomerName, COALESCE(COUNT(od.OrderNum), 0) AS TotalOrders FROM OrderDetails AS od RIGHT JOIN Payment AS p ON od.OrderNum = p.OrderNum JOIN PaymentProfiles AS pp ON p.PaymentProfileID = pp.PaymentProfileID RIGHT JOIN Customers AS c ON pp.CustomerID = c.CustomerID GROUP BY c.CustomerID, c.CustomerName HAVING TotalOrders > {total_orders};"
        query_desc = "Query Description: Customers with more than a particular number of orders and their total orders"
        input_orders_label.pack()
        input_orders.pack()
    elif query_choice == "4":
        validity_period = input_validity.get()
        if not validity_period:
            messagebox.showwarning("Missing Input", "Please enter the validity period.")
            return
        query = f"SELECT m.MembershipName, COUNT(*) AS TotalOrders FROM OrderDetails AS od JOIN Payment AS p ON od.OrderNum = p.OrderNum JOIN PaymentProfiles AS pp ON p.PaymentProfileID = pp.PaymentProfileID JOIN Customers AS c ON pp.CustomerID = c.CustomerID JOIN MembershipType AS m ON c.MembershipTypeID = m.MembershipTypeID WHERE c.MembershipTypeID IN (SELECT MembershipTypeID FROM MembershipType WHERE ValidityPeriod > {validity_period}) GROUP BY m.MembershipName;"
        query_desc = "Query Description: Total orders by membership type with a validity period over a specific year period"
        input_validity_label.pack()
        input_validity.pack()
    elif query_choice == "5":
        customer_ID = input_customer_ID.get()
        if not customer_ID:
            messagebox.showwarning("Missing Input", "Please enter the Customer ID.")
            return
        query = f"SELECT c.CustomerID, c.CustomerName,od.ProductID, p.ProductName, od.ProductQuantity FROM OrderDetails od JOIN Product p ON od.ProductID = p.ProductID JOIN ( SELECT ps.OrderNum, c.CustomerName, c.CustomerID FROM Payment ps JOIN PaymentProfiles pp ON ps.PaymentProfileID = pp.PaymentProfileID  JOIN Customers c ON pp.CustomerID = c.CustomerID WHERE c.CustomerID = {customer_ID} ) AS c ON od.OrderNum = c.OrderNum;"
        query_desc = "Query Description: Retrieve Product Details for Orders Placed by CustomerID"
        input_customer_ID_label.pack()
        input_customer_ID.pack()
    elif query_choice == "6":
        delivery_status = input_delivery_status.get()
        if not delivery_status:
            messagebox.showwarning("Missing Input", "Please enter the Delivery Status as 1 for Successful or 0 for Unsuccessful")
            return
        query = f"SELECT od.ProductID, p.ProductName, SUM(od.ProductQuantity) AS TotalProductQuantity, GROUP_CONCAT(c.CustomerID) AS CustomerIDs, GROUP_CONCAT(c.CustomerName) AS CustomerNames,MAX(d.DeliverySuccess) AS DeliverySuccess,MAX(d.DeliveryDate) AS LastDeliveryDate FROM OrderDetails od INNER JOIN Payment ps ON od.OrderNum = ps.OrderNum INNER JOIN Delivery d ON ps.OrderNum = d.OrderNum INNER JOIN Product p ON od.ProductID = p.ProductID JOIN Customers c ON d.CustomerID = c.CustomerID WHERE d.DeliverySuccess ={delivery_status} GROUP BY od.ProductID, p.ProductName ORDER BY od.ProductID ASC;"
        query_desc = "Query Description: Retrieve top 30 products Ordered by Customers Who Received Deliveries Successful or Unsuccessful ordered by Product ID"
        input_delivery_status_label.pack()
        input_delivery_status.pack()

    try:
        cursor.execute(query)
        columns = cursor.description
        results = cursor.fetchall()
        if results:
            headers = [column[0] for column in columns]
            data = [headers] + [list(row) for row in results]
            display_results(headers, data, query_desc)
            export_button.config(state=NORMAL)
            export_pdf_button.config(state=NORMAL)
        else:
            messagebox.showinfo("No results", "No results found.")
            export_button.config(state=DISABLED)
            export_pdf_button.config(state=DISABLED)
    except mysql.connector.Error as e:
        messagebox.showerror("Error", f"Error executing query:\n{e}")



# Function to display query results in Treeview
def display_results(headers, data, query_desc):
    result_tree.delete(*result_tree.get_children()) 
    

    result_tree.column("#0", width=50)

    result_tree["columns"] = headers
    for header in headers:
        result_tree.heading(header, text=header)


    for index, row in enumerate(data, start=0):
        if index != 0:  # Skip the first row
            result_tree.insert("", "end",values=row)

    query_desc_label.config(text=query_desc)



def export_to_excel(event=None):
    try:
        data = []
        headers = result_tree["columns"]
        for child in result_tree.get_children():
            values = result_tree.item(child)["values"]
            data.append(values)
        
        df = pd.DataFrame(data, columns=headers)
        df.to_excel("output.xlsx", index=False)
        messagebox.showinfo("Export Successful", "Data has been exported to output.xlsx in your current working directory")
    except Exception as e:
        messagebox.showerror("Export Error", f"An error occurred while exporting data:\n{e}")


def export_to_pdf(event=None):
    try:
        data = []
        headers = result_tree["columns"]
        for child in result_tree.get_children():
            values = result_tree.item(child)["values"]
            data.append(values)

      
        pdf_filename = "output.pdf"
        pdf = SimpleDocTemplate(pdf_filename, pagesize=letter)
        
      
        table_data = [headers] + data
        table = Table(table_data)

       
        style = TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                            ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                            ('GRID', (0, 0), (-1, -1), 1, colors.black)])
        table.setStyle(style)

    
        pdf.build([table])

        messagebox.showinfo("Export Successful", f"Data has been exported to {pdf_filename} in your current working directory")
    except Exception as e:
        messagebox.showerror("Export Error", f"An error occurred while exporting data:\n{e}")


def update_input_fields(*args):
    query_choice = query_var.get()
    query_desc_label.config(text=query_descriptions.get(query_choice, ""))
    result_tree.delete(*result_tree.get_children())  
    result_tree["columns"] = ()
    result_tree.heading("#0", text="")
    if query_choice == "1":
        input_date_label.pack(anchor=W)
        input_date.pack(anchor=W)
        input_date.focus_set()  
        input_amount_label.pack_forget()
        input_amount.pack_forget()
        input_orders_label.pack_forget()
        input_orders.pack_forget()
        input_validity_label.pack_forget()
        input_validity.pack_forget()
        input_customer_ID_label.pack_forget()
        input_customer_ID.pack_forget()
        input_delivery_status_label.pack_forget()
        input_delivery_status.pack_forget()
    elif query_choice == "2":
        input_amount_label.pack(anchor=W)
        input_amount.pack(anchor=W)
        input_amount.focus_set()  
        input_date_label.pack_forget()
        input_date.pack_forget()
        input_orders_label.pack_forget()
        input_orders.pack_forget()
        input_validity_label.pack_forget()
        input_validity.pack_forget()
        input_customer_ID_label.pack_forget()
        input_customer_ID.pack_forget()
        input_delivery_status_label.pack_forget()
        input_delivery_status.pack_forget()
    elif query_choice == "3":
        input_orders_label.pack(anchor=W)
        input_orders.pack(anchor=W)
        input_orders.focus_set() 
        input_date_label.pack_forget()
        input_date.pack_forget()
        input_amount_label.pack_forget()
        input_amount.pack_forget()
        input_validity_label.pack_forget()
        input_validity.pack_forget()
        input_customer_ID_label.pack_forget()
        input_customer_ID.pack_forget()
        input_delivery_status_label.pack_forget()
        input_delivery_status.pack_forget()

    elif query_choice == "4":
        input_validity_label.pack(anchor=W)
        input_validity.pack(anchor=W)
        input_validity.focus_set()  
        input_date_label.pack_forget()
        input_date.pack_forget()
        input_amount_label.pack_forget()
        input_amount.pack_forget()
        input_orders_label.pack_forget()
        input_orders.pack_forget()
        input_customer_ID_label.pack_forget()
        input_customer_ID.pack_forget()
        input_delivery_status_label.pack_forget()
        input_delivery_status.pack_forget()
    elif query_choice == "5":
        input_customer_ID_label.pack(anchor=W)
        input_customer_ID.pack(anchor=W)
        input_customer_ID.focus_set()  
        input_date_label.pack_forget()
        input_date.pack_forget()
        input_amount_label.pack_forget()
        input_amount.pack_forget()
        input_orders_label.pack_forget()
        input_orders.pack_forget()
        input_validity_label.pack_forget()
        input_validity.pack_forget()
        input_delivery_status_label.pack_forget()
        input_delivery_status.pack_forget()
    elif query_choice == "6":
        input_delivery_status_label.pack(anchor=W)
        input_delivery_status.pack(anchor=W)
        input_delivery_status.focus_set()  
        input_date_label.pack_forget()
        input_date.pack_forget()
        input_amount_label.pack_forget()
        input_amount.pack_forget()
        input_orders_label.pack_forget()
        input_orders.pack_forget()
        input_validity_label.pack_forget()
        input_validity.pack_forget()
        input_customer_ID_label.pack_forget()
        input_customer_ID.pack_forget()

    export_button.config(state=DISABLED)
    export_pdf_button.config(state=DISABLED)
   
    execute_button.pack(side=BOTTOM, anchor=W)


try:
    user = input("Enter MySQL username: ")

    if user == 'mm_team03_02':
        host = 'CSSQL'
    else:
        host = input("Enter MySQL host: ")

    password = getpass.getpass("Enter MySQL password: ")
    database = input("Enter MySQL database name: ")


    connection = mysql.connector.connect(
        user=user,
        password=password,
        host=host,
        database=database
    )
    if connection.is_connected():
        print("Connected to MySQL database.")
        cursor = connection.cursor()

        root = Tk()
        root.title("Milestone-3 Team3")
        
        label_font = font.Font(family="Helvetica", size=12)
        button_font = font.Font(family="Helvetica", size=12, weight="bold")
        text_font = font.Font(family="Courier", size=12)

        message_label = Label(root, text="Please select a query number from the below dropdown:", font=label_font)
        message_label.pack(anchor=W)

  
        query_descriptions = {
            "1": "Query Description: Total deliveries and success rate for a specific date with delivery agent",
            "2": "Query Description: Total products returned by customers with payments over a specific amount",
            "3": "Query Description: Customers with more than a particular number of orders and their total orders",
            "4": "Query Description: Total orders by membership type with a validity period over a specific year period",
            "5": "Query Description: Retrieve Product Details for Orders Placed by CustomerID",
            "6": "Query Description: Retrieve top 30 products Ordered by Customers Who Received Deliveries Successful or Unsuccessful ordered by Product ID"
        }
        query_var = StringVar()
        query_var.set("1")  
        query_var.trace("w", update_input_fields)  
        query_menu = OptionMenu(root, query_var, *query_descriptions.keys())
        query_menu.pack(anchor=W)
        query_desc_label = Label(root, text="", font=label_font)
        query_desc_label.pack(anchor=W)

        input_date_label = Label(root, text="Enter delivery date (YYYY-MM-DD): ex. (2023-08-19,2023-08-20)", font=label_font)
        input_date = Entry(root, font=text_font)
        input_amount_label = Label(root, text="Enter a specific amount: ex. (100,150)", font=label_font)
        input_amount = Entry(root, font=text_font)
        input_orders_label = Label(root, text="Enter the amount of orders: (Please enter number between 1-8)", font=label_font)
        input_orders = Entry(root, font=text_font)
        input_validity_label = Label(root, text="Enter the validity period: (Please enter 1 or 2 for results)", font=label_font)
        input_validity = Entry(root, font=text_font)
        input_customer_ID_label = Label(root, text="Enter the CustomerID: ex. (525,123)", font=label_font)
        input_customer_ID = Entry(root, font=text_font)
        input_delivery_status_label = Label(root, text="Enter the Delivery Status as 1 for successful or 0 for unsuccessful deliveries:", font=label_font)
        input_delivery_status = Entry(root, font=text_font)


      
        execute_button = Button(root, text="Execute Query", command=execute_query, font=button_font)
        
    
        result_frame = Frame(root)
        result_frame.pack(expand=True, fill=BOTH)

       
        result_tree = ttk.Treeview(result_frame)
        result_tree.pack(side=LEFT, expand=True, fill=BOTH)
        

        y_scrollbar = ttk.Scrollbar(result_frame, orient=VERTICAL, command=result_tree.yview)
        y_scrollbar.pack(side=RIGHT, fill=Y)
        y_scrollbar.config(style="Vertical.TScrollbar")
        result_tree.configure(yscrollcommand=y_scrollbar.set)


        x_scrollbar = ttk.Scrollbar(root, orient=HORIZONTAL, command=result_tree.xview)
        x_scrollbar.pack(side=BOTTOM, fill=X)
        result_tree.configure(xscrollcommand=x_scrollbar.set)
        

        export_button = Button(root, text="Export to Excel", command=export_to_excel, font=button_font)
        export_button.pack(side=BOTTOM, anchor=W)
        

        export_pdf_button = Button(root, text="Export to PDF", command=export_to_pdf, font=button_font)
        export_pdf_button.pack(side=BOTTOM, anchor=W)


        export_button.bind("<Button-1>", export_to_excel)

        window_width = 800
        window_height = 600
        root.geometry(f"{window_width}x{window_height}")


        label_font = font.Font(family="Helvetica", size=12)
        button_font = font.Font(family="Helvetica", size=12, weight="bold")
        text_font = font.Font(family="Courier", size=12)


        y_scrollbar = ttk.Scrollbar(result_frame, orient=VERTICAL, command=result_tree.yview)
        y_scrollbar.pack(side=RIGHT, fill=Y)


        y_scrollbar.config(style="Vertical.TScrollbar")


        result_tree.configure(yscrollcommand=y_scrollbar.set)

        x_scrollbar = ttk.Scrollbar(root, orient=HORIZONTAL, command=result_tree.xview)
        x_scrollbar.pack(side=BOTTOM, fill=X)

        result_tree.configure(xscrollcommand=x_scrollbar.set)

        update_input_fields()
        root.mainloop()

except mysql.connector.Error as e:
    messagebox.showerror("Error", f"Error connecting to MySQL database:\n{e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("\nMySQL connection closed.")
